# Lab: Metaclass

Write a metaclass that allows us to use `class` syntax to create a `dict`, e.g.

```python
class MyDict(metaclass=Dictify):
    a = 1
    b = 2
```

creates the dict `{'a': 1, 'b': 2}`

Hint: Metaclasses and class decorators are not required to return classes!

In [2]:
class Dictify(type):
    def __new__(meta, name, bases, dct):
#         return type.__new__(meta, name, bases, dct)
#         return super().__new__(meta, name, bases, dct)
        return {
            k: v for k, v in dct.items()
            if not k.startswith('_')
        }
    
class MyDict(metaclass=Dictify):
    a = 1
    b = 2
    c = 'foo'
    
MyDict

{'a': 1, 'b': 2, 'c': 'foo'}

In [3]:
def dictify(name, bases, dct):
    return {
        k: v for k, v in dct.items()
        if not k.startswith('_')
    }

class MyDict2(metaclass=dictify):
    a = 1
    b = 2
    c = 'foo'
    
MyDict2

{'a': 1, 'b': 2, 'c': 'foo'}

In [4]:
def dictify(name, bases, dct):
    return dct

In [9]:
class SomeDictionaryIWantToUseAClassToCreate(metaclass=dictify):
    'docstring'
    a: 1
    develop='intelligence'

In [10]:
SomeDictionaryIWantToUseAClassToCreate

{'__module__': '__main__',
 '__qualname__': 'SomeDictionaryIWantToUseAClassToCreate',
 '__annotations__': {'a': 1},
 '__doc__': 'docstring',
 'develop': 'intelligence'}

Step 2: use a class decorator to do the same

In [11]:
def dictify_decorator(cls):
    dct = {
        key: getattr(cls, key)
        for key in dir(cls)
        if not key.startswith('_')
#         if not callable(getattr(cls, key))
    }
#     dct = {
#         key: value
#         for key, value in cls.__dict__.items()
#         if not callable(value)
#     }    
    return dct

In [12]:
@dictify_decorator
class MyDict:
    a = 5
    b = 10
MyDict

{'a': 5, 'b': 10}

In [13]:
class MyDict2:
    a = 5
    b = 10

In [14]:
type(MyDict2)

type

In [15]:
tmp0 = dictify_decorator(MyDict2)
type(tmp0)

dict

In [16]:
MyDict2 = tmp0
   
MyDict2

{'a': 5, 'b': 10}

In [17]:
type(MyDict2)

dict

# Lab: Descriptor

Write a (non-data) descriptor that will allow us to calculate an attribute value the *first* time it is loaded, and cache it for subsequent loads.

In [18]:
class reify:
    def __init__(self, getter):
        self._getter = getter
        self._attrname = None
        
    def __set_name__(self, cls, name):
        self._attrname = name
        
    def __get__(self, instance, type_):
        #print('Calling the descriptor')
        if instance is None:
            return self
        value = self._getter(instance)
        instance.__dict__[self._attrname] = value
        return value

class MyClass:
    
    @reify
    def a(self):
        print('Calculate a!')
        return 'a'

In [19]:
obj = MyClass()
obj.a

Calculate a!


'a'

In [20]:
obj.a

'a'

Implement the descriptor above as a *data* descriptor:

In [21]:
class reify():
    missing = object()
    
    def __init__(self, getter):
        self._getter = getter
        self._attrname = None
        
    def __set_name__(self, cls, name):
        self._attrname = name
        
    def __get__(self, instance, type):        
        #print('Calling the descriptor', self, instance, type)
        if instance is None:
            return self
        # Check to see if the value is in the instance dict
        value = instance.__dict__.get(self._attrname, self.missing)
        if value is self.missing:
            value = self._getter(instance)
            instance.__dict__[self._attrname] = value
        return value
    
    def __set__(self, instance, value):
        raise TypeError

class MyClassData():
    
    @reify
    def a(self):
        print('Calculate a!')
        return 'a'
    
#     a = reify(a)

In [22]:
obj = MyClassData()
obj.a

Calculate a!


'a'

In [23]:
obj.a

'a'

In [24]:
obj.a = 5

TypeError: 

In [25]:
obj1 = MyClassData()
obj1.a

Calculate a!


'a'

In [26]:
obj2 = MyClass()
obj2.a

Calculate a!


'a'

Data descriptor

In [27]:
%timeit obj1.a

243 ns ± 3.58 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Non-data descriptor

In [28]:
%timeit obj2.a

43.7 ns ± 0.551 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
